In [1]:
# importing relevant packages 

import torch 
from torchvision import models
import pandas as pd
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import glob
import os
import cv2
import matplotlib.pyplot as plt 
from matplotlib import pyplot
import torchvision.transforms as transforms
from torchsummary import summary
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import json
from PIL import Image
import random
import albumentations as A
import glob
from shapely.geometry import Polygon

/home/dell/Desktop/xxxx/Uni/Kirigami_project/Keypoint_detection_notebooks/.Keypoint_detection_notebooks/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
class VGG_model(nn.Module):
    def __init__(self):
        super(VGG_model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
        
        self.avgpool = nn.Sequential(
            nn.Conv2d(512,512, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512,50, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.AdaptiveAvgPool2d(output_size=(8,8))
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(3200, 300),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Dropout(0.3),
            nn.Linear(300, 32),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
        

In [3]:
model = VGG_model()
model.load_state_dict(torch.load('./Model_VGG_7_1000_17300_epoch'))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
model.eval()

VGG_model(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.1, inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): LeakyReLU(negative_slope=0.1, inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.1, inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): LeakyReLU(negative_slope=0.1, inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): LeakyReLU(negative_slope=0.1, inplace=True)
    (14): Conv2d(

In [4]:
def predict_keypoints(img):
    img = cv2.resize(img, (224,224))
    img_tensor = transforms.ToTensor()(img)
    img_mean = img_tensor.mean(dim = (1,2))
    img_std = img_tensor.std(dim = (1,2))
        
    img_normalised = transforms.Normalize(img_mean, img_std)(img_tensor)
    img_normalised = img_normalised.to(device)
    
    key_points = model(img_normalised[None]).flatten().detach().cpu().numpy()
    
    return key_points

In [5]:
def plot_keypoints(img, keypoints):                                                             

    plt.imshow(img)

    x_points = keypoints[0::2]
    y_points = keypoints[1::2]
    plt.scatter(x_points*img.shape[1], y_points*img.shape[0], s = 4, c=(1,0,0))
    plt.show()

In [6]:
def process_num(x):
    return x*random.uniform(0.96, 1.04)

In [12]:
dataset_names = ['Grasp_dataset_banana', 'Grasp_dataset_chicken', 'Grasp_dataset_greengrape', 'Grasp_dataset_redgrape', 'Grasp_dataset_tomato', 'Grasp_dataset_ibuprofen']

header = ['weight_reading_1', 'weight_reading_2', 'pressure_reading_1', 'pressure_reading_2', 'force_reading_1', 'force_reading_2', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y', 'label']

current_dir = os. getcwd()

weight_reading_1_arr = []
weight_reading_2_arr = []
pressure_reading_1_arr = []
pressure_reading_2_arr = []
force_reading_1_arr = []
force_reading_2_arr = []

p1_x_arr = []
p1_y_arr = []
p2_x_arr = []
p2_y_arr = []
p3_x_arr = []
p3_y_arr = []
p4_x_arr = []
p4_y_arr = []
p5_x_arr = []
p5_y_arr = []
p6_x_arr = []
p6_y_arr = []
p7_x_arr = []
p7_y_arr = []
p8_x_arr = []
p8_y_arr = []
p9_x_arr = []
p9_y_arr = []
p10_x_arr = []
p10_y_arr = []
p11_x_arr = []
p11_y_arr = []
p12_x_arr = []
p12_y_arr = []
p13_x_arr = []
p13_y_arr = []
p14_x_arr = []
p14_y_arr = []
p15_x_arr = []
p15_y_arr = []
p16_x_arr = []
p16_y_arr = []
label_arr = []


for f in dataset_names:
    
    image_dir = os.path.join(current_dir + '/Grasp_dataset_5_test', f)
    data_dir = os.path.join(image_dir, f + '.csv')
    df = pd.read_csv(data_dir)
    weight_reading_1_arr = np.concatenate((weight_reading_1_arr, df['weight_reading_1'].values))
    weight_reading_2_arr = np.concatenate((weight_reading_2_arr, df['weight_reading_2'].values))
    pressure_reading_1_arr = np.concatenate((pressure_reading_1_arr, df['pressure_reading_1'].values))
    pressure_reading_2_arr = np.concatenate((pressure_reading_2_arr, df['pressure_reading_2'].values))
    force_reading_1_arr = np.concatenate((force_reading_1_arr, df['force_reading_1'].values))
    force_reading_2_arr = np.concatenate((force_reading_2_arr, df['force_reading_2'].values))
    label_arr = np.concatenate((label_arr, df['label'].values))
    
    j = 0
    
    while j < df.shape[0]:
        image = cv2.imread(os.path.join(image_dir, df.iloc[j]['image_name']))
        keypoints = predict_keypoints(image)
        
#         plot_keypoints(image, keypoints)
        
#         if j == 20:
#             plot_keypoints(image, keypoints)
        

#         plt.imshow(image)

#         plt.scatter(kirigami_x_points*image.shape[1], kirigami_y_points*image.shape[0], s = 4, c=(1,0,0))
#         plt.show()
        
        p1_x_arr.append(keypoints[0])
        p1_y_arr.append(keypoints[1])
        p2_x_arr.append(keypoints[2])
        p2_y_arr.append(keypoints[3])
        p3_x_arr.append(keypoints[4])
        p3_y_arr.append(keypoints[5])
        p4_x_arr.append(keypoints[6])
        p4_y_arr.append(keypoints[7])
        p5_x_arr.append(keypoints[8])
        p5_y_arr.append(keypoints[9])
        p6_x_arr.append(keypoints[10])
        p6_y_arr.append(keypoints[11])
        p7_x_arr.append(keypoints[12])
        p7_y_arr.append(keypoints[13])
        p8_x_arr.append(keypoints[14])
        p8_y_arr.append(keypoints[15])
        p9_x_arr.append(keypoints[16])
        p9_y_arr.append(keypoints[17])
        p10_x_arr.append(keypoints[18])
        p10_y_arr.append(keypoints[19])
        p11_x_arr.append(keypoints[20])
        p11_y_arr.append(keypoints[21])
        p12_x_arr.append(keypoints[22])
        p12_y_arr.append(keypoints[23])
        p13_x_arr.append(keypoints[24])
        p13_y_arr.append(keypoints[25])
        p14_x_arr.append(keypoints[26])
        p14_y_arr.append(keypoints[27])
        p15_x_arr.append(keypoints[28])
        p15_y_arr.append(keypoints[29])
        p16_x_arr.append(keypoints[30])
        p16_y_arr.append(keypoints[31])
        
        j = j + 1
        
#         if j == 2:
#             break
#     break



p1_x_arr = np.array(p1_x_arr) - np.array(p1_x_arr)
p2_x_arr = np.array(p2_x_arr) - np.array(p1_x_arr)
p3_x_arr = np.array(p3_x_arr) - np.array(p1_x_arr)
p4_x_arr = np.array(p4_x_arr) - np.array(p1_x_arr)
p5_x_arr = np.array(p5_x_arr) - np.array(p1_x_arr)
p6_x_arr = np.array(p6_x_arr) - np.array(p1_x_arr)
p7_x_arr = np.array(p7_x_arr) - np.array(p1_x_arr)
p8_x_arr = np.array(p8_x_arr) - np.array(p1_x_arr)
p9_x_arr = np.array(p9_x_arr) - np.array(p1_x_arr)
p10_x_arr = np.array(p10_x_arr) - np.array(p1_x_arr)
p11_x_arr = np.array(p11_x_arr) - np.array(p1_x_arr)
p12_x_arr = np.array(p12_x_arr) - np.array(p1_x_arr)
p13_x_arr = np.array(p13_x_arr) - np.array(p1_x_arr)
p14_x_arr = np.array(p14_x_arr) - np.array(p1_x_arr)
p15_x_arr = np.array(p15_x_arr) - np.array(p1_x_arr)
p16_x_arr = np.array(p16_x_arr) - np.array(p1_x_arr)

p1_y_arr = np.array(p1_y_arr) - np.array(p1_y_arr)
p2_y_arr = np.array(p2_y_arr) - np.array(p1_y_arr)
p3_y_arr = np.array(p3_y_arr) - np.array(p1_y_arr)
p4_y_arr = np.array(p4_y_arr) - np.array(p1_y_arr)
p5_y_arr = np.array(p5_y_arr) - np.array(p1_y_arr)
p6_y_arr = np.array(p6_y_arr) - np.array(p1_y_arr)
p7_y_arr = np.array(p7_y_arr) - np.array(p1_y_arr)
p8_y_arr = np.array(p8_y_arr) - np.array(p1_y_arr)
p9_y_arr = np.array(p9_y_arr) - np.array(p1_y_arr)
p10_y_arr = np.array(p10_y_arr) - np.array(p1_y_arr)
p11_y_arr = np.array(p11_y_arr) - np.array(p1_y_arr)
p12_y_arr = np.array(p12_y_arr) - np.array(p1_y_arr)
p13_y_arr = np.array(p13_y_arr) - np.array(p1_y_arr)
p14_y_arr = np.array(p14_y_arr) - np.array(p1_y_arr)
p15_y_arr = np.array(p15_y_arr) - np.array(p1_y_arr)
p16_y_arr = np.array(p16_y_arr) - np.array(p1_y_arr)


weight_diff_1_arr = np.array(weight_reading_1_arr) - np.array(force_reading_1_arr)
weight_diff_2_arr = np.array(force_reading_2_arr) - np.array(weight_reading_2_arr) 

grasp_dataset = pd.concat([pd.Series(weight_diff_1_arr), pd.Series(weight_diff_2_arr), pd.Series(pressure_reading_1_arr), pd.Series(pressure_reading_2_arr), pd.Series(force_reading_1_arr), pd.Series(force_reading_2_arr), pd.Series(p1_x_arr), pd.Series(p1_y_arr), pd.Series(p2_x_arr), pd.Series(p2_y_arr), pd.Series(p3_x_arr), pd.Series(p3_y_arr), pd.Series(p4_x_arr), pd.Series(p4_y_arr), pd.Series(p5_x_arr), pd.Series(p5_y_arr), pd.Series(p6_x_arr), pd.Series(p6_y_arr), pd.Series(p7_x_arr), pd.Series(p7_y_arr), pd.Series(p8_x_arr), pd.Series(p8_y_arr), pd.Series(p9_x_arr), pd.Series(p9_y_arr), pd.Series(p10_x_arr), pd.Series(p10_y_arr), pd.Series(p11_x_arr), pd.Series(p11_y_arr), pd.Series(p12_x_arr), pd.Series(p12_y_arr), pd.Series(p13_x_arr), pd.Series(p13_y_arr), pd.Series(p14_x_arr), pd.Series(p14_y_arr), pd.Series(p15_x_arr), pd.Series(p15_y_arr), pd.Series(p16_x_arr), pd.Series(p16_y_arr), pd.Series(label_arr)], axis=1, keys=header)

# num_cols = grasp_dataset.select_dtypes(include=['float']).columns
# df[num_cols] = grasp_dataset[num_cols].applymap(process_num)

# combined_df_set = grasp_dataset
# combined_df_set = pd.concat([combined_df_set, grasp_dataset], axis=0)


In [13]:
pd.options.display.max_columns = None
print(grasp_dataset.head(3))

  weight_reading_1 weight_reading_2 pressure_reading_1 pressure_reading_2  \
0             94.7             74.5               57.1               81.8   
1             93.6             76.0               49.8               89.4   
2             85.0             85.3               71.4               62.5   

  force_reading_1 force_reading_2  p1_x  p1_y      p2_x      p2_y      p3_x  \
0           381.1           380.4   0.0   0.0  0.476145  0.148061  0.801756   
1           405.3           404.9   0.0   0.0  0.478944  0.140528  0.801249   
2           424.5           424.4   0.0   0.0  0.485395  0.140352  0.796633   

       p3_y      p4_x      p4_y      p5_x      p5_y      p6_x      p6_y  \
0  0.347107  0.800586  0.701951  0.474357  0.920719  0.176802  0.683752   
1  0.357484  0.799978  0.688327  0.478358  0.925050  0.175786  0.690301   
2  0.372899  0.797963  0.665095  0.487538  0.931645  0.169806  0.711425   

       p7_x      p7_y      p8_x      p8_y      p9_x      p9_y     p10_x  

In [14]:
grasp_dataset

,weight_reading_1,weight_reading_2,pressure_reading_1,pressure_reading_2,force_reading_1,force_reading_2,p1_x,p1_y,p2_x,p2_y,p3_x,p3_y,p4_x,p4_y,p5_x,p5_y,p6_x,p6_y,p7_x,p7_y,p8_x,p8_y,p9_x,p9_y,p10_x,p10_y,p11_x,p11_y,p12_x,p12_y,p13_x,p13_y,p14_x,p14_y,p15_x,p15_y,p16_x,p16_y,label
0,94.7,74.5,57.1,81.8,381.1,380.4,0.0,0.0,0.476145,0.148061,0.801756,0.347107,0.800586,0.701951,0.474357,0.920719,0.176802,0.683752,0.467663,0.267111,0.459117,0.399312,0.457155,0.534973,0.463240,0.664019,0.469351,0.793111,0.497483,0.790960,0.515026,0.658433,0.521799,0.530210,0.513710,0.410357,0.498568,0.277107,chicken
1,93.6,76.0,49.8,89.4,405.3,404.9,0.0,0.0,0.478944,0.140528,0.801249,0.357484,0.799978,0.688327,0.478358,0.925050,0.175786,0.690301,0.466643,0.264053,0.457081,0.397329,0.454978,0.533093,0.461692,0.661923,0.469142,0.791954,0.498418,0.789508,0.514059,0.656354,0.520588,0.527496,0.512706,0.406539,0.498935,0.272271,chicken
2,85.0,85.3,71.4,62.5,424.5,424.4,0.0,0.0,0.485395,0.140352,0.796633,0.372899,0.797963,0.665095,0.487538,0.931645,0.169806,0.711425,0.464823,0.266716,0.450898,0.399854,0.447424,0.533179,0.455603,0.659745,0.467924,0.792234,0.499503,0.791692,0.510573,0.657470,0.515440,0.527015,0.508162,0.404357,0.497947,0.271845,chicken
3,95.7,73.5,95.2,78.4,400.5,399.7,0.0,0.0,0.478393,0.143902,0.803673,0.353880,0.801926,0.703479,0.472802,0.934190,0.178017,0.680005,0.468065,0.266662,0.458320,0.399514,0.456544,0.535773,0.463254,0.665390,0.468778,0.795150,0.498097,0.791739,0.517077,0.658080,0.525100,0.531220,0.516674,0.413069,0.500692,0.279129,chicken
4,93.6,75.5,69.7,61.9,382.6,382.0,0.0,0.0,0.478158,0.149013,0.800823,0.365384,0.801498,0.716471,0.473579,0.926071,0.179364,0.679256,0.466257,0.264541,0.457349,0.396416,0.456169,0.532687,0.463189,0.662189,0.468829,0.791961,0.497681,0.792286,0.515036,0.659661,0.521733,0.531556,0.513048,0.410763,0.498223,0.277242,chicken
5,93.8,75.7,53.2,71.1,381.3,380.7,0.0,0.0,0.480637,0.143967,0.798962,0.368025,0.799668,0.694122,0.477278,0.923647,0.177112,0.686543,0.466972,0.262741,0.456359,0.395261,0.454696,0.530917,0.462008,0.659847,0.469368,0.789603,0.497430,0.789170,0.513549,0.656554,0.519888,0.528269,0.511621,0.408202,0.498120,0.274975,chicken
6,93.2,76.0,55.7,76.2,374.8,373.7,0.0,0.0,0.479255,0.149778,0.800940,0.355672,0.800416,0.689452,0.478511,0.920352,0.177356,0.689037,0.466842,0.265329,0.457292,0.396872,0.455781,0.532215,0.462744,0.660547,0.469884,0.789623,0.498420,0.788697,0.513645,0.656528,0.519784,0.527814,0.511943,0.407066,0.498522,0.274621,chicken
7,89.3,80.5,57.7,85.3,380.0,379.2,0.0,0.0,0.481922,0.150292,0.797830,0.369688,0.799902,0.679679,0.484098,0.923908,0.173977,0.696700,0.464579,0.266526,0.453203,0.398150,0.451153,0.532262,0.459130,0.659742,0.469168,0.790700,0.499208,0.791083,0.511129,0.658678,0.515871,0.528489,0.508184,0.405718,0.496884,0.274097,chicken
8,91.4,78.2,103.4,100.4,404.4,403.8,0.0,0.0,0.482817,0.140637,0.799536,0.366684,0.800010,0.678586,0.482188,0.928687,0.174563,0.691467,0.465605,0.262885,0.452958,0.396080,0.450870,0.530963,0.458924,0.658862,0.468905,0.789950,0.499091,0.789825,0.512999,0.656339,0.518927,0.527111,0.511054,0.405756,0.498732,0.272384,chicken
9,95.4,74.0,68.6,76.6,384.6,384.2,0.0,0.0,0.477005,0.139218,0.801169,0.350008,0.799590,0.702400,0.474144,0.918329,0.178828,0.682909,0.467047,0.260761,0.458574,0.394028,0.456955,0.530138,0.463301,0.658680,0.469019,0.787124,0.497337,0.785692,0.514810,0.653840,0.521778,0.525913,0.513567,0.405370,0.498715,0.271068,chicken


In [ ]:
# grasp_dataset.to_csv('/home/dell/Desktop/xxxx/Uni/Kirigami_project/Keypoint_detection_notebooks/Grasp_dataset_3/' + 'validation_df.csv', index=False)

In [15]:
# grasp_data = grasp_dataset.drop(columns=['label', 'weight_reading_1', 'weight_reading_2'], axis=1)
grasp_data = grasp_dataset.drop(columns=['label'], axis=1)
# grasp_data = grasp_dataset.drop(columns=['label', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y'], axis=1)
grasp_data['weight_reading_1'] = grasp_data['weight_reading_1'] / 1000
grasp_data['weight_reading_2'] = grasp_data['weight_reading_2'] / 1000
grasp_data['pressure_reading_1'] = grasp_data['pressure_reading_1'] / 100
grasp_data['pressure_reading_2'] = grasp_data['pressure_reading_2'] / 100
grasp_data['force_reading_1'] = grasp_data['force_reading_1'] / 1000
grasp_data['force_reading_2'] = grasp_data['force_reading_2'] / 1000
grasp_label = grasp_dataset['label']

print(grasp_data.head(2))
print(grasp_label.head(2))

  weight_reading_1 weight_reading_2 pressure_reading_1 pressure_reading_2  \
0           0.0947           0.0745              0.571              0.818   
1           0.0936            0.076              0.498              0.894   

  force_reading_1 force_reading_2  p1_x  p1_y      p2_x      p2_y      p3_x  \
0          0.3811          0.3804   0.0   0.0  0.476145  0.148061  0.801756   
1          0.4053          0.4049   0.0   0.0  0.478944  0.140528  0.801249   

       p3_y      p4_x      p4_y      p5_x      p5_y      p6_x      p6_y  \
0  0.347107  0.800586  0.701951  0.474357  0.920719  0.176802  0.683752   
1  0.357484  0.799978  0.688327  0.478358  0.925050  0.175786  0.690301   

       p7_x      p7_y      p8_x      p8_y      p9_x      p9_y     p10_x  \
0  0.467663  0.267111  0.459117  0.399312  0.457155  0.534973  0.463240   
1  0.466643  0.264053  0.457081  0.397329  0.454978  0.533093  0.461692   

      p10_y     p11_x     p11_y     p12_x     p12_y     p13_x     p13_y  \
0  

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

data_train, data_test, label_train, label_test = train_test_split(grasp_data, grasp_label, test_size=0.2, random_state=np.random.randint(100))

# print(X_train.head(3))

# clf = KNeighborsClassifier(n_neighbors=7)

# knn.fit(X_train, y_train)

# y_pred = knn.predict(X_test)

# accuracy = knn.score(X_test, y_test)
# print('Accuracy:', accuracy)
# clf = RandomForestClassifier(n_estimators=1000, max_depth=50, random_state=np.random.randint(100))
clf = RandomForestClassifier(n_estimators=1000, max_depth=100, random_state=np.random.randint(100), n_jobs = -1)
# clf = HistGradientBoostingClassifier(max_iter=200, random_state=np.random.randint(100))
# clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=1, random_state=0)
# clf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='hard')

# num_folds = 10
# cv_method = KFold(n_splits=num_folds, shuffle=True, random_state=np.random.randint(100))
# cv_results = cross_val_score(clf, grasp_data, grasp_label, cv=cv_method, scoring='accuracy')

# print('Cross-validation results:', cv_results)
# print('Average accuracy:', cv_results.mean())

clf.fit(data_train, label_train)
label_pred = clf.predict(data_test)
cm = confusion_matrix(label_test, label_pred)

# short_dataset_names = ['chicken', 'tomato']


# cm_df = pd.DataFrame(cm, index=short_dataset_names, columns=short_dataset_names)

# print(cm_df)
# print(cm)

accuracy = clf.score(data_test, label_test)
print("Accuracy:", accuracy)

Accuracy: 0.9833333333333333


In [16]:
label_pred = clf.predict(grasp_data)
print(label_pred)
accuracy = clf.score(grasp_data, grasp_label)
print("Accuracy:", accuracy)

['chicken' 'chicken' 'chicken' 'chicken' 'chicken' 'chicken' 'chicken'
 'chicken' 'chicken' 'chicken' 'redgrape' 'redgrape' 'redgrape' 'redgrape'
 'redgrape' 'redgrape' 'redgrape' 'redgrape' 'greengrape' 'redgrape'
 'redgrape' 'redgrape' 'redgrape' 'redgrape' 'redgrape' 'redgrape'
 'redgrape' 'redgrape' 'redgrape' 'redgrape' 'tomato' 'tomato' 'tomato'
 'tomato' 'tomato' 'tomato' 'tomato' 'tomato' 'tomato' 'tomato'
 'ibuprofen' 'ibuprofen' 'ibuprofen' 'ibuprofen' 'ibuprofen' 'ibuprofen'
 'ibuprofen' 'ibuprofen' 'ibuprofen' 'ibuprofen']
Accuracy: 0.82


In [ ]:
Chicken, egg, tomato